# Data Wrangling with MongoDB - Project 3

I choose the Raleigh city in North Carolina to work on for this project on data wrangeling with MongoDB.

##  Import modules to start with parsing the data

In [1]:
# Importing necessary python modules
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import collections
import pymongo

In [2]:
osm_file = open("raleigh_north-carolina.osm",'r')

## Data Audit

Let us parse through the data using ElementTree iteratevly to see how many unique elements there are in the dataset to an overview understanding of the data.  

In [3]:
# parsing through the dataset iteratvely to find unique elements in the dataset
def count_tags(filename):
        tags = {}
        for event, elem in ET.iterparse(filename):
            if elem.tag in tags: 
                tags[elem.tag] += 1
            else:
                tags[elem.tag] = 1
        return tags


In [4]:
# Call the function count-tags on the Chicago dataset
tags_raleigh = count_tags('raleigh_north-carolina.osm')
pprint.pprint(tags_raleigh)

{'bounds': 1,
 'member': 8684,
 'nd': 2622994,
 'node': 2325555,
 'osm': 1,
 'relation': 889,
 'tag': 886324,
 'way': 238794}


Before  processing the data and add it into a database, we should check the "k" value for each "<tag>" (there are 886,324 tag elements in the dataset as showen in the above result) and see if there are any potential problems.
We define  'key_type' & 'process_map' functions that check the "k" value for each tag.The 'key_type'provides a count of each of three tag categories in a dictionary: "lower", for tags that contain only lowercase letters and are valid, "lower_colon", for valid tags with a colon in their names, "problemchars", for tags with problematic characters.

In [5]:

import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


In [6]:
raleigh_keys = process_map('raleigh_north-carolina.osm')
pprint.pprint(raleigh_keys)

{'lower': 552793, 'lower_colon': 290466, 'other': 43063, 'problemchars': 2}


Let us look at the number of  unique users that have contributed to the city map of Raleigh,NC.The result below diplay that there are 952 uniques users who has contributed to this. 


In [7]:
#people invovlved in the map editing.
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for ele in element:
            if 'uid' in ele.attrib:
                users.add(ele.attrib['uid'])
    return users
users = process_map('raleigh_north-carolina.osm')
len(users)

952

# Probelms in the Dataset

## Audit Street Names

Most of the problem in this data set is name inconsitency of street addresses. They street names consits of inconsitent abberevations. The following code use regular expression to match the very last of the street name as most street names end with street types like Avenue, Street, Road etc. And adds the street names that is not conistent with most comment street names which are listed in 'expected' list below to a set-'street_types'.


In [8]:
from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive", "Lane", "Parkway", 
                         "Place", "Road", "Square", "Street", "Trail"]

mapping = {'Ave'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Dr.'  : 'Drive',
           'Blvd.': 'Boulevard',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'Pky'  : 'Parkway',
           'Rd'   : 'Road',
           'Rd.'   : 'Road',
           'St'   : 'Street',
           'St,'  : 'Street',
           'street' :"Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'Sq'   : "Square"}

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

- audit_street_type function search the input string for the regex. If there is a match and it is not within the "expected" list, add the match as a key and add the string to the set.
- is_street_name function looks at the attribute k if k="addre:street"
- audit functio will return the list that match previous two functions. After that, we would do a pretty print the output of the audit. With the list of all the abbreviated street types we can understand and fill-up our "mapping" dictionary as a preparatio to convert these street name into proper form.

In [9]:
streettypes_raleigh = audit('raleigh_north-carolina.osm')

We then take a brief look at the unclean street name using pretty print below


In [10]:
pprint.pprint(dict(streettypes_raleigh))

{'100': set(['100', 'Westgate Park Dr #100']),
 '1000': set(['Six Forks Road #1000']),
 '102': set(['Page Rd #102']),
 '1100': set(['Fayetteville St #1100']),
 '112': set(['Lynn Road #112']),
 '115': set(['Brier Creek Pkwy #115']),
 '17': set(['US Highway 17']),
 '206': set(['Barrett Dr Suite 206']),
 '27604-2428': set(['27604-2428']),
 '27609': set(['27609']),
 '27609-5360': set(['27609-5360']),
 '315': set(['Kildaire Farm Road #315']),
 '501': set(['US 15;US 501']),
 '54': set(['Highway 54',
            'State Highway 54',
            'West Highway 54',
            'West NC Highway 54',
            'West State Highway 54']),
 '55': set(['Highway 55', 'NC Highway 55', 'US 55']),
 '70': set(['US 70']),
 '751': set(['NC 751', 'NC Highway 751']),
 'Alley': set(['Yates Motor Company Alley']),
 'Ave': set(['Atlantic Ave',
             'E. Winmore Ave',
             'East Winmore Ave',
             'Glenwood Ave',
             'Mountford Ave',
             'N Harrison Ave',
             'No

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
This last function update_name is the last step of the process, which take the old name and update them with a better name

In [13]:
def update_name(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

    return name

for street_type, ways in streettypes_raleigh.iteritems():
    for name in ways:
        better_name = update_name(name, mapping, street_type_re)
        print name, "=>", better_name
if name == 'Blue Ridge Rd':
    assert better_name == 'Blue Ridge Road'
if name == 'Chapel Hill Rd':
    assert better_name == 'Chapel Hill Road'        

Oldham Forest Crossing => Oldham Forest Crossing
The Circle at North Hills => The Circle at North Hills
Morris St, => Morris Street
NC Highway 55 West => NC Highway 55 West
Highway 54 West => Highway 54 West
Highway 55 West => Highway 55 West
Highway West => Highway West
Columbia Place West => Columbia Place West
E Rosemary St. => E Rosemary St.
East Franklin St. => East Franklin St.
W. Franklin St. => W. Franklin St.
West Rosemary St. => West Rosemary St.
East Corcoran St. => East Corcoran St.
W Rosemary St. => W Rosemary St.
W. Pettigrew St. => W. Pettigrew St.
Lynn Road #112 => Lynn Road #112
Kildaire Farm Road #315 => Kildaire Farm Road #315
Buck Jones Rd => Buck Jones Road
Falls of Neuse Rd => Falls of Neuse Road
Chapel Hill Rd => Chapel Hill Road
Blue Ridge Rd => Blue Ridge Road
Six Forks Rd => Six Forks Road
N Roxboro Rd => N Roxboro Road
Trinity Rd => Trinity Road
Creedmoor Rd => Creedmoor Road
SW Maynard Rd => SW Maynard Road
27609 => 27609
Chapel Hill => Chapel Hill
Katahdin 

## Audit Zip code

Using part of the above codes let check zipcodes for uniformity or consitency. We can see from the result that the zipcode formatting is inconsistent. There are zipcodes which are not in the area that start with '26', some zipcodes have only 4 digits, the other zipcodes have formats like '27513-3507' and '275198404' which show inconsistency in the zipcode formatting. 

In [12]:
# Let check zipcode for inconsistency 
from collections import defaultdict

def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]
    
    if not twoDigits.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)
    
    elif twoDigits != 27:
        invalid_zipcodes[twoDigits].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

zipcode_raleigh = audit_zip('raleigh_north-carolina.osm')
pprint.pprint(dict(zipcode_raleigh))

{'26': set(['2612-6401']),
 '27': set(['27069-5439',
            '27162-2750',
            '27162-3449',
            '27162-3452',
            '27162-3454',
            '27278',
            '27502',
            '27510',
            '27511',
            '27511-4701',
            '27511-4702',
            '27511-4734',
            '27511-4735',
            '27511-4762',
            '27511-4791',
            '27511-5525',
            '27511-5927',
            '27511-5928',
            '27511-9998',
            '27513',
            '27513-3507',
            '27513-3510',
            '27513-3574',
            '27513-3595',
            '27513-3601',
            '27513-3611',
            '27513-4278',
            '27513-8143',
            '27513-8446',
            '27513-8480',
            '27513-8720',
            '27514',
            '27514-2885',
            '27514-3307',
            '27514-7415',
            '27516',
            '27517',
            '27518',
            '27519',
         

Below is the output of the updated zipcode, some have 5 digits, other have 4 digits which needs updating. 

In [14]:
# updating zipcode 

def update_name(zipcode):
    testNum = re.findall('[a-zA-Z]*', zipcode)
    if testNum:
        testNum = testNum[0]
    testNum.strip()
    if testNum == "CA":
        updatedzcode = (re.findall(r'\d+', zipcode))
        if updatedzcode:
            if updatedzcode.__len__() == 2:
                return (re.findall(r'\d+', zipcode))[0] + "-" +(re.findall(r'\d+', zipcode))[1]
            else:
                return (re.findall(r'\d+', zipcode))[0]

for street_type, ways in zipcode_raleigh.iteritems():
    for name in ways:
        better_name = update_name(name)
        print name, "=>", better_name

NC => None
2612-6401 => None
27609-5938 => None
27609-5939 => None
27612-2201 => None
27612-2200 => None
27609-5934 => None
27609-5935 => None
27609-5936 => None
27609-5937 => None
27609-5930 => None
27609-5931 => None
27609-5932 => None
27609-5933 => None
27609-5859 => None
27609-5853 => None
27609-5852 => None
27609-5850 => None
27609-5857 => None
27609-5856 => None
27609-4678 => None
27612-6302 => None
27604-2549 => None
27612-6304 => None
27612-6305 => None
27604-2543 => None
27604-2540 => None
27604-2547 => None
27604-2545 => None
27604-2544 => None
27604-2612 => None
27603-1725 => None
27603-1724 => None
27609-5589 => None
27609-5581 => None
27609-5580 => None
27609-5582 => None
27609-5619 => None
27609-5618 => None
27609-5615 => None
27609-5614 => None
27609-5617 => None
27609-5616 => None
27609-5611 => None
27609-5610 => None
27609-5613 => None
27609-5612 => None
27609-4528 => None
27609-4529 => None
27609-4524 => None
27609-4525 => None
27609-4526 => None
27609-4527 => None
27

# Preparing for DataBase-MongoDB 
To store the data into a mongodb database, it could be converted from XML to JSON format. We need to do the following to convert the data from XML format to JSON.

- you should process only 2 types of top level tags: "node" and "way"
- all attributes of "node" and "way" should be turned into regular key/value pairs, except:
    - attributes in the CREATED array should be added under a key "created"
    - attributes for latitude and longitude should be added to a "pos" array,
      for use in geospacial indexing. Make sure the values inside "pos" array are floats
      and not strings. 
- if the second level tag "k" value contains problematic characters, it should be ignored
- if the second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
- if the second level tag "k" value does not start with "addr:", but contains ":", you can
  process it in a way that you feel is best. For example, you might split it into a two-level
  dictionary like with "addr:", or otherwise convert the ":" to create a valid key.
- if there is a second ":" that separates the type/direction of a street,
  the tag should be ignored, for example:




In [16]:
import re
import codecs
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
address_regex = re.compile(r'^addr\:')
street_regex = re.compile(r'^street')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        # YOUR CODE HERE
        node['type'] = element.tag
        # let us assign an empty dict to address
        address = {}
        # parse through attributes of all elements
        for attr  in element.attrib:
            if attr in CREATED:
                if 'created' not in node:
                    node['created'] = {}
                node['created'][attr] = element.get(attr)
            elif attr in ['lat', 'lon']:
                continue
            else:
                node[attr] = element.get(attr)
        # finding lattitute and logitude positions
        if 'lat' in element.attrib and 'lon' in element.attrib:
            node['pos'] = [float(element.get('lat')), float(element.get('lon'))]

        # parse second-level tags for nodes
        for e in element:
            # parse second-level tags for ways and populate `node_refs`
            if e.tag == 'nd':
                if 'node_refs' not in node:
                    node['node_refs'] = []
                if 'ref' in e.attrib:
                    node['node_refs'].append(e.get('ref'))

            # throw out not-tag elements and elements without `k` or `v`
            if e.tag != 'tag' or 'k' not in e.attrib or 'v' not in e.attrib:
                continue
            key = e.get('k')
            val = e.get('v')

            # skip problematic characters
            if problemchars.search(key):
                continue

            # parse address k-v pairs
            elif address_regex.search(key):
                key = key.replace('addr:', '')
                address[key] = val

            # catch-all
            else:
                node[key] = val
        # compile address
        if len(address) > 0:
            node['address'] = {}
            street_full = None
            street_dict = {}
            street_format = ['prefix', 'name', 'type']
            # parse through address objects
            for key in address:
                val = address[key]
                if street_regex.search(key):
                    if key == 'street':
                        street_full = val
                    elif 'street:' in key:
                        street_dict[key.replace('street:', '')] = val
                else:
                    node['address'][key] = val
            # assign street_full or fallback to compile street dict
            if street_full:
                node['address']['street'] = street_full
            elif len(street_dict) > 0:
                node['address']['street'] = ' '.join([street_dict[key] for key in street_format])
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data


In [17]:
# Print the JSON data type
process_map('raleigh_north-carolina.osm',False)


[{'created': {'changeset': '7525913',
   'timestamp': '2011-03-11T18:03:13Z',
   'uid': '38487',
   'user': 'jumbanho',
   'version': '4'},
  'id': '21592692',
  'pos': [35.9746754, -78.9086172],
  'type': 'node'},
 {'created': {'changeset': '6118030',
   'timestamp': '2010-10-20T17:35:46Z',
   'uid': '38487',
   'user': 'jumbanho',
   'version': '2'},
  'id': '21595910',
  'pos': [35.9323309, -78.9249163],
  'type': 'node'},
 {'created': {'changeset': '9059469',
   'timestamp': '2011-08-18T19:00:34Z',
   'uid': '398883',
   'user': 'FIM',
   'version': '3'},
  'id': '21595911',
  'pos': [35.9293805, -78.9272018],
  'type': 'node'},
 {'created': {'changeset': '9059469',
   'timestamp': '2011-08-18T19:00:34Z',
   'uid': '398883',
   'user': 'FIM',
   'version': '3'},
  'id': '21595918',
  'pos': [35.9291511, -78.9249346],
  'type': 'node'},
 {'created': {'changeset': '9059469',
   'timestamp': '2011-08-18T19:00:34Z',
   'uid': '398883',
   'user': 'FIM',
   'version': '3'},
  'id': '215

## Data Overview with MongoDB

I have used the command below to insert the json data into MongoDB in my local computer 



In [57]:
# mongoimport -h 127.0.0.1:27017 --db  openstreet_raleigh  --collection  collection  --file  raleigh_north-carolina.osm.json


Now let us access the stored dataset from mongodb and do different data analysis. To do that first we need to connect to the MongoDB server installed locally.

In [58]:
from pymongo import MongoClient
client = MongoClient('localhost:27017')
db = client.openstreet_raleigh

### Checking the size of the file

In [64]:

import os
print 'The size of the origanal dataset {} MB'.format(os.path.getsize('raleigh_north-carolina.osm')/1.0e6) # convert from bytes to megabytes
print 'The size of the json dataset {} MB'.format(os.path.getsize('raleigh_north-carolina.osm.json')/1.0e6) # convert from bytes to megabytes

The size of the origanal dataset 483.230968 MB
The size of the json dataset 558.674601 MB


### How many documents in the dataset ?

The result show that there are 2564349 documnets after processing the data

In [67]:
# let us access the json file from the collection in the database
raleigh_data_json = db.collection

In [68]:
raleigh_data_json.find().count()

2564349

### The number of nodes and ways in the dataset

In [69]:
print "Number of nodes:",raleigh_data_json.find({'type':'node'}).count()
print "Number of ways:",raleigh_data_json.find({'type':'way'}).count()

Number of nodes: 2325552
Number of ways: 238789


### Who contributes the most?
Below is the list of the top ten contributors

In [78]:

result = raleigh_data_json.aggregate( [
                                        { "$group" : {"_id" : "$created.user", 
                                        "count" : { "$sum" : 1} } },
                                        { "$sort" : {"count" : -1} }, 
                                        { "$limit" : 10 } ] )

print(list(result))

[{u'count': 1557194, u'_id': u'jumbanho'}, {u'count': 202705, u'_id': u'JMDeMai'}, {u'count': 129883, u'_id': u'bdiscoe'}, {u'count': 114022, u'_id': u'woodpeck_fixbot'}, {u'count': 103432, u'_id': u'bigal945'}, {u'count': 66743, u'_id': u'yotann'}, {u'count': 41462, u'_id': u'runbananas'}, {u'count': 32414, u'_id': u'sandhill'}, {u'count': 30731, u'_id': u'MikeInRaleigh'}, {u'count': 21942, u'_id': u'Clay Hobbs'}]


### List of top 20 cuisine in Raleigh

In [79]:

cuisine = raleigh_data_json.aggregate([{"$match":{"amenity":{"$exists":1},
                                 "amenity":"restaurant",}},      
                      {"$group":{"_id":{"Food":"$cuisine"},
                                 "count":{"$sum":1}}},
                      {"$project":{"_id":0,
                                  "Food":"$_id.Food",
                                  "Count":"$count"}},
                      {"$sort":{"Count":-1}}, 
                      {"$limit":20}])
print(list(cuisine))

[{u'Food': None, u'Count': 206}, {u'Food': u'american', u'Count': 44}, {u'Food': u'mexican', u'Count': 42}, {u'Food': u'pizza', u'Count': 40}, {u'Food': u'italian', u'Count': 30}, {u'Food': u'burger', u'Count': 27}, {u'Food': u'chinese', u'Count': 27}, {u'Food': u'sandwich', u'Count': 17}, {u'Food': u'japanese', u'Count': 16}, {u'Food': u'asian', u'Count': 16}, {u'Food': u'regional', u'Count': 15}, {u'Food': u'indian', u'Count': 10}, {u'Food': u'seafood', u'Count': 8}, {u'Food': u'thai', u'Count': 8}, {u'Food': u'greek', u'Count': 6}, {u'Food': u'barbecue', u'Count': 6}, {u'Food': u'steak_house', u'Count': 6}, {u'Food': u'sushi', u'Count': 4}, {u'Food': u'vietnamese', u'Count': 3}, {u'Food': u'french', u'Count': 3}]


### List of top 10 post code in Raleigh

In [72]:

postcode = raleigh_data_json.aggregate( [ 
    { "$match" : { "address.postcode" : { "$exists" : 1} } }, 
    { "$group" : { "_id" : "$address.postcode", "count" : { "$sum" : 1} } },  
    { "$sort" : { "count" : -1}},
      {"$limit":10}] )
print(list(postcode))

[{u'count': 1612, u'_id': u'27560'}, {u'count': 904, u'_id': u'27519'}, {u'count': 721, u'_id': u'27609'}, {u'count': 687, u'_id': u'27701'}, {u'count': 526, u'_id': u'27705'}, {u'count': 432, u'_id': u'27615'}, {u'count': 328, u'_id': u'27510'}, {u'count': 236, u'_id': u'27604'}, {u'count': 190, u'_id': u'27513'}, {u'count': 182, u'_id': u'27514'}]


In [75]:

building = raleigh_data_json.aggregate([
       {'$match': {'building': { '$exists': 1}}}, 
        {'$group': {'_id': '$building',
                    'count': {'$sum': 1}}}, 
        {'$sort': {'count': -1}},
        {'$limit': 10}])
print(list(building))

[{u'count': 128500, u'_id': u'yes'}, {u'count': 9757, u'_id': u'house'}, {u'count': 9502, u'_id': u'residential'}, {u'count': 8631, u'_id': u'shed'}, {u'count': 566, u'_id': u'apartments'}, {u'count': 344, u'_id': u'detached'}, {u'count': 325, u'_id': u'commercial'}, {u'count': 190, u'_id': u'retail'}, {u'count': 159, u'_id': u'roof'}, {u'count': 136, u'_id': u'school'}]


## Other ideas 

While auditing the street names and zipcodes in the xml dataset we have wittnessed that there are a lot of street naming inconsistncies , there are also problems in the zipcode formats ,some zipcodes are not in the area, some zipcodes contain only 4 digits and others have five and with extetnsions. This could be problems that arise due to the different bodys which have contributed to the openstreet dataset. So to alleviate this problems I would suggest the following:-

- To establish a standardized data inputting or reporting format that should  be comminicated to to any contributor of the dataset, such as commenly used street names and zipcodes
- Use other data sources such as google API to augument with the data we accessed from openstreetmap which would solve missing data or also validates the present dataset.
- Ecouraging as many people or entities to contribute to the data, as we saw above in the top contributors list, only few people or entities are contributing, the top one contributes about 1557194 and the second one contibutes about 202705, which show a big difference in contibution to the dataset. 

#  Conclusions

For this project I used data from www.openstreetmap.org for Raleigh, NC. The  dataset  was iteratelvely parsed to find out the total number of  tags contained in the XML documents. Then  I did auditing on streetname to find out if the names are consitent all over the whole document. I have found out that streetnames are not used consitently, for example 'St.', 'street'and 'St,' are used to for 'Street' , and 'Dr'/'Dr.' used for 'Drive' and soon. Later these names are upadated to the most common namings. This problem in naming inconsistency might be caused due to a number of contributor for the dataset.  After auditig and updating of the dataset was converted from XML to JSON format and imported into MongoDB data base. I have faced challenge to import into the mongodb database using the 'insert()' method of pymongo until I able to import the json data using the 'mongoimport' method direclty in the command window. Finally some data analysis was made by connecting to MongoDB and accessing the stored json data from mongodb database.
           


# Reference 

1. [http://wiki.openstreetmap.org/wiki/OSM_XML](http://wiki.openstreetmap.org/wiki/OSM_XML)
2. [https://www.w3schools.com/xml/xml_whatis.asp](https://www.w3schools.com/xml/xml_whatis.asp)
3. [https://stackoverflow.com/questions/15171622/mongoimport-of-json-file](https://stackoverflow.com/questions/15171622/mongoimport-of-json-file)
4. [https://stackoverflow.com/questions/9805451/how-to-find-names-of-all-collections-using-pymongo](https://stackoverflow.com/questions/9805451/how-to-find-names-of-all-collections-using-pymongo)